# DATA Gathering  From API , GlassDoor ,Kaggle  

In [ ]:
# USA similar for  INDIA  and UK 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

job_roles = ['Data Analyst', 'Data Scientist', 'Data Engineer', 'Power BI Developer']
base_url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.keyword='

all_jobs = []

for role in job_roles:
    print(f"\nScraping jobs for: {role}")
    query = role.replace(" ", "%20")
    driver.get(base_url + query)
    time.sleep(3)

    count = 0
    while count < 60:
        jobs = driver.find_elements(By.CLASS_NAME, 'JobCard_jobCardContainer__arQlW')

        for j in jobs:
            if count >= 60:
                break
            try:
                company = j.find_element(By.CLASS_NAME,'EmployerProfile_compactEmployerName__9MGcV').text
            except:
                company = 'N/A'

            try:
                title = j.find_element(By.CLASS_NAME,'JobCard_jobTitle__GLyJ1').text
            except:
                title = 'N/A'

            try:
                location = j.find_element(By.CLASS_NAME,'JobCard_location__Ds1fM').text
            except:
                location = 'N/A'

            try:
                salary = j.find_element(By.CLASS_NAME,'JobCard_salaryEstimate__QpbTW').text
            except:
                salary = 'N/A'

            try:
                update = j.find_element(By.CLASS_NAME,'JobCard_listingAge__jJsuc').text
            except:
                update = 'N/A'

            all_jobs.append({
                'Job Role Searched': role,
                'Company Name': company,
                'Job Title': title,
                'Location': location,
                'Salary': salary,
                'Last Update': update
            })
            count += 1

        # Try to click 'Load More'
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test='load-more']"))
            )
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)
        except:
            print("No more pages.")
            break

# Save to CSV
df = pd.DataFrame(all_jobs)
df.to_csv("jobs_data.csv", index=False)
print("\nScraping complete. Data saved to 'jobs_data.csv'.")

driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)

job_roles = ['Data Analyst', 'Data Scientist', 'Data Engineer', 'Power BI Developer']
base_url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword='

all_jobs = []

for role in job_roles:
    print(f"\nScraping jobs for: {role}")
    query = role.replace(" ", "%20")
    driver.get(base_url + query)
    time.sleep(5)

    count = 0
    while count < 60:
        jobs = driver.find_elements(By.CLASS_NAME, 'JobCard_jobCardContainer__arQlW')

        for j in jobs:
            if count >= 60:
                break
            try:
                company = j.find_element(By.CLASS_NAME,'EmployerProfile_compactEmployerName__9MGcV').text
            except:
                company = 'N/A'

            try:
                title = j.find_element(By.CLASS_NAME,'JobCard_jobTitle__GLyJ1').text
            except:
                title = 'N/A'

            try:
                location = j.find_element(By.CLASS_NAME,'JobCard_location__Ds1fM').text
            except:
                location = 'N/A'

            try:
                salary = j.find_element(By.CLASS_NAME,'JobCard_salaryEstimate__QpbTW').text
            except:
                salary = 'N/A'

            try:
                update = j.find_element(By.CLASS_NAME,'JobCard_listingAge__jJsuc').text
            except:
                update = 'N/A'

            all_jobs.append({
                'Job Role Searched': role,
                'Company Name': company,
                'Job Title': title,
                'Location': location,
                'Salary': salary,
                'Last Update': update
            })
            count += 1

        # Try to click 'Load More'
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test='load-more']"))
            )
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)
        except:
            print("No more pages or load more button not found.")
            break

# Save to CSV
df = pd.DataFrame(all_jobs)
df.to_csv("glassdoor_usa_jobs.csv", index=False)
print("\n✅ Scraping complete. Data saved to 'glassdoor_usa_jobs.csv'.")

driver.quit()


In [6]:
#From API 

In [ ]:
import requests
import pandas as pd
import time

# Your JSearch API key
API_KEY = "2be3b5b8f8msh9f90dd72488b3e0p131250jsna50049762712"

headers = {
    "X-RapidAPI-Key": API_KEY,
    "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

job_roles = ["data analyst", "data scientist", "data engineer", "power bi developer"]
countries = ["India", "Germany", "United Kingdom", "United States"]

all_jobs = []

for role in job_roles:
    for country in countries:
        print(f"🔍 Fetching jobs for '{role}' in {country}...")
        query = f"{role} in {country}"
        page = 1
        max_pages = 5

        while page <= max_pages:
            url = "https://jsearch.p.rapidapi.com/search"
            params = {
                "query": query,
                "page": str(page),
                "num_pages": "1"
            }

            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                jobs = data.get("data", [])
                if not jobs:
                    break

                for job in jobs:
                    posted_date = job.get("job_posted_at_datetime_utc")
                    try:
                        year = pd.to_datetime(posted_date).year if posted_date else "Unknown"
                    except:
                        year = "Unknown"

                    job_info = {
                        "Job Title": job.get("job_title"),
                        "Company": job.get("employer_name"),
                        "City": job.get("job_city"),
                        "Country": country,
                        "Experience (Months)": job.get("job_required_experience", {}).get("required_experience_in_months"),
                        "Job Description": job.get("job_description"),
                        "Skills Required": job.get("job_required_skills"),
                        "Salary Range": (
                            f"{job.get('job_salary_currency')} {job.get('job_salary', {}).get('salary_min')} - {job.get('job_salary', {}).get('salary_max')}"
                            if job.get("job_salary") else "N/A"
                        ),
                        "Job Type": job.get("job_employment_type"),
                        "Posted Date": posted_date,
                        "Year": year
                    }

                    # Optional: Print job year for debugging
                    print(f"🗓️ Job Year: {year}")

                    all_jobs.append(job_info)

                page += 1
                time.sleep(1)
            else:
                print(f"❌ Failed to fetch data: {response.status_code}")
                break

# Save to DataFrame
df = pd.DataFrame(all_jobs)

# Save to CSV
df.to_csv("global_job_data_with_city_and_year.csv", index=False)
print("✅ Job data (with year info) saved to 'global_job_data_with_city_and_year.csv'.")


  #  Merging all  file in one to get enough data  for the analysis in excel  and the final  file is :

In [7]:
final_raw_file=pd.read_csv("ds_salaries.csv")

In [8]:
final_raw_file.sample(5)

,work_year,employment_type,job_title,salary,salary_currency,company_location,country,Code,Country Name
2178,2023,FT,Data Engineer,129000,USD,US,United States,NaN,NaN
330,2022,FT,Data Analyst,112900,USD,US,United States,NaN,NaN
4495,2025,FT,Power BI Developer,51000,GBP,GB,United Kingdom,NaN,NaN
2642,2022,FT,Data Engineer,135000,USD,US,United States,NaN,NaN
1383,2023,FT,AI Developer,80000,USD,SE,NaN,NaN,NaN


#  As  the Data is focussing on the Job Demand Market analysids of jobs related  Data  mainly FRom  INDIA ,USA,UK   i  Mappp the data  , Normalized  it 

In [4]:
import pandas as pd
import random
from datetime import datetime

# Load the dataset
file_path = "ds_salaries.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Clean column names
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

# Remove rows where 'work_year' is not a valid year (non-digit, like 'Year')
df = df[df["work_year"].str.isdigit()]
df["work_year"] = df["work_year"].astype(int)

# Define skill sets based on job title keywords
skill_map = {
    "data scientist": ["Python", "Machine Learning", "Statistics", "SQL"],
    "data analyst": ["Excel", "SQL", "Tableau", "Power BI"],
    "machine learning": ["Python", "TensorFlow", "Scikit-learn", "Deep Learning"],
    "data engineer": ["Python", "Spark", "SQL", "ETL"],
    "business analyst": ["Excel", "Power BI", "Communication", "SQL"],
    "research scientist": ["Python", "R", "Statistics", "Data Analysis"],
    "software engineer": ["Java", "Python", "System Design", "Git"],
    "ai": ["Python", "Deep Learning", "Neural Networks", "NLP"]
}


# Function to infer skills based on job title
def infer_skills(title):
    title = title.lower()
    skills = set()
    for keyword, skill_list in skill_map.items():
        if keyword in title:
            skills.update(skill_list)
    if not skills:
        skills = {"Python", "SQL", "Communication"}  # Default general skills
    return ", ".join(sorted(skills))

# Define industry mapping based on job titles
industry_map = {
    "data scientist": "Technology",
    "data analyst": "Finance",
    "machine learning": "Technology",
    "data engineer": "Technology",
    "business analyst": "Consulting",
    "research scientist": "Healthcare",
    "software engineer": "Technology",
    "ai": "Technology"
}

# Function to infer industry
def infer_industry(title):
    title = title.lower()
    for keyword, industry in industry_map.items():
        if keyword in title:
            return industry
    return "General"

# Generate realistic posting date
def generate_posting_date(year):
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # Valid for all months
    return datetime(year, month, day)

# Determine season from month
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

# Add synthetic data
df["skills"] = df["job_title"].apply(infer_skills)
df["industry"] = df["job_title"].apply(infer_industry)
df["posting_date"] = df["work_year"].apply(generate_posting_date)
df["season"] = df["posting_date"].dt.month.apply(get_season)
df["quarter"] = df["posting_date"].dt.quarter

# (Optional) Save the enhanced dataset
df.to_csv("enhanced_ds_salaries.csv", index=False)


In [5]:
df.head()

,work_year,employment_type,job_title,salary,salary_currency,company_location,country,code,country_name,skills,industry,posting_date,season,quarter
0,2020,FT,Data Scientist,70000,EUR,DE,Germany,US,United States,"Machine Learning, Python, SQL, Statistics",Technology,2020-06-04,Summer,2
1,2020,FT,Machine Learning Scientist,260000,USD,JP,Japan,IN,India,"Deep Learning, Python, Scikit-learn, TensorFlow",Technology,2020-09-26,Autumn,3
2,2020,FT,Big Data Engineer,85000,GBP,GB,United Kingdom,GB,United Kingdom,"ETL, Python, SQL, Spark",Technology,2020-04-06,Spring,2
3,2020,FT,Product Data Analyst,20000,USD,HN,NaN,DE,Germany,"Excel, Power BI, SQL, Tableau",Finance,2020-04-10,Spring,2
4,2020,FT,Machine Learning Engineer,150000,USD,US,United States,FR,France,"Deep Learning, Python, Scikit-learn, TensorFlow",Technology,2020-08-24,Summer,3


In [9]:
api_df = pd.read_csv("global_job_data_with_city2.csv")


In [11]:
# Step-by-step transformation of the API dataset

# 1. Rename and standardize columns
api_df_cleaned = api_df.rename(columns={
    'Job Title': 'job_title',
    'Company': 'company',
    'City': 'city',
    'Country': 'company_location',
    'Experience (Months)': 'experience_months',
    'Skills Required': 'skills',
    'Job Type': 'employment_type',
    'Posted Date': 'posting_date',
    'Salary Range': 'salary'
})

# 2. Convert posting_date to datetime
api_df_cleaned['posting_date'] = pd.to_datetime(api_df_cleaned['posting_date'], errors='coerce')

# 3. Add season and quarter
api_df_cleaned['season'] = api_df_cleaned['posting_date'].dt.month % 12 // 3 + 1
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
api_df_cleaned['season'] = api_df_cleaned['season'].map(season_map)
api_df_cleaned['quarter'] = api_df_cleaned['posting_date'].dt.quarter

# 4. Convert experience months into experience level
def get_experience_level(months):
    if pd.isnull(months):
        return 'unknown'
    elif months <= 24:
        return 'entry-level'
    elif months <= 60:
        return 'mid-level'
    elif months <= 120:
        return 'senior-level'
    else:
        return 'executive'

api_df_cleaned['experience_level'] = api_df_cleaned['experience_months'].apply(get_experience_level)

# 5. Define skill_map to infer skills
skill_map = {
    "data scientist": ["Python", "Machine Learning", "Statistics", "SQL"],
    "data analyst": ["Excel", "SQL", "Tableau", "Power BI"],
    "machine learning": ["Python", "TensorFlow", "Scikit-learn", "Deep Learning"],
    "data engineer": [
        "Python", "SQL", "Apache Spark", "ETL Development", "Airflow",
        "Data Warehousing", "Kafka", "NoSQL", "Cloud Platforms"
    ],
    "business analyst": ["Excel", "Power BI", "Communication", "SQL"],
    "research scientist": ["Python", "R", "Statistics", "Data Analysis"],
    "software engineer": ["Java", "Python", "System Design", "Git"],
    "ai": ["Python", "Deep Learning", "Neural Networks", "NLP"],
    "power bi developer": [
        "Power BI", "DAX", "Power Query", "SQL", "Excel",
        "ETL Tools", "Data Warehousing", "Data Modeling"
    ]
}

# 6. Infer skills from job title if not present
def infer_skills(job_title):
    if pd.isnull(job_title):
        return []
    job_title = job_title.lower()
    for key in skill_map:
        if key in job_title:
            return skill_map[key]
    return []

api_df_cleaned['skills'] = api_df_cleaned.apply(
    lambda row: row['skills'] if pd.notnull(row['skills']) else infer_skills(row['job_title']),
    axis=1
)

# 7. Infer industry from job title
def infer_industry(job_title):
    if pd.isnull(job_title):
        return 'Unknown'
    job_title = job_title.lower()
    if 'data' in job_title:
        return 'Data & AI'
    elif 'software' in job_title or 'developer' in job_title:
        return 'Technology'
    elif 'analyst' in job_title:
        return 'Consulting'
    elif 'engineer' in job_title:
        return 'Engineering'
    elif 'research' in job_title:
        return 'Research'
    else:
        return 'Other'

api_df_cleaned['industry'] = api_df_cleaned['job_title'].apply(infer_industry)
api_df_cleaned.to_csv("enhanced_API_job_salaries.csv", index=False)
# Display cleaned data (head only)
api_df_cleaned.head()


,job_title,company,city,company_location,experience_months,Job Description,skills,salary,employment_type,posting_date,season,quarter,experience_level,industry
0,Modelling & Data Analyst- India GDC (Gurugram),ERM Group,NaN,India,NaN,Who is ERM?\nERM is a leading global sustainab...,"[Excel, SQL, Tableau, Power BI]",NaN,Full-time,2025-04-24 09:00:00+00:00,Spring,2.0,unknown,Data & AI
1,"Analyst, Data Governance",WARNERMEDIA,Hyderabad,India,NaN,Welcome to Warner Bros. Discovery… the stuff d...,[],NaN,Full-time,2025-03-25 00:00:00+00:00,Spring,1.0,unknown,Data & AI
2,Senior Data Analyst,eBay,Bengaluru,India,NaN,"At eBay, we're more than a global ecommerce le...","[Excel, SQL, Tableau, Power BI]",NaN,Full-time,2025-04-11 00:00:00+00:00,Spring,2.0,unknown,Data & AI
3,Manager-Data Analytics,American Express,Gurugram,India,NaN,You Lead the Way. We’ve Got Your Back.\n\nWith...,[],NaN,Full-time,2025-04-24 00:00:00+00:00,Spring,2.0,unknown,Data & AI
4,Data Scientist,United Airlines,Gurugram,India,NaN,Achieving our goals starts with supporting you...,"[Python, Machine Learning, Statistics, SQL]",NaN,Full-time,2025-04-24 23:00:00+00:00,Spring,2.0,unknown,Data & AI


In [12]:
#  merging  both  data   from glassdoor 

In [15]:
# Full code to normalize, transform, and merge the two datasets

# Step 1: Load both datasets again
api_data = pd.read_csv("enhanced_API_job_salaries.csv")
ds_data = pd.read_csv("enhanced_ds_salaries.csv")

# Step 2: Standardize column names (lowercase and underscores)
api_data.columns = api_data.columns.str.lower().str.replace(" ", "_")
ds_data.columns = ds_data.columns.str.lower().str.replace(" ", "_")

# Step 3: Drop or fill unnecessary columns in API data
api_data = api_data.drop(columns=[
    'city', 'company', 'country', 'country_name', 'code',
    'job_description', 'salary_currency'
], errors='ignore')

# Step 4: Add missing columns to API data
if 'work_year' not in api_data.columns and 'posting_date' in api_data.columns:
    api_data['posting_date'] = pd.to_datetime(api_data['posting_date'], errors='coerce')
    api_data['work_year'] = api_data['posting_date'].dt.year

# Step 5: Ensure both have the same column order and names
common_cols = set(api_data.columns).intersection(set(ds_data.columns))
api_data_aligned = api_data[list(common_cols)].copy()
ds_data_aligned = ds_data[list(common_cols)].copy()

# Step 6: Combine both datasets
merged_data = pd.concat([ds_data_aligned, api_data_aligned], ignore_index=True)

# Final check - show shape and first few rows
merged_data.shape, merged_data.head()
merged_data.to_csv("Data_job.csv", index=False)

In [17]:
df=pd.read_csv("Data_job.csv")

In [19]:
df.shape

(5228, 10)

In [20]:
df.head()

,work_year,salary,employment_type,industry,job_title,season,company_location,skills,quarter,posting_date
0,2020.0,70000,FT,Technology,Data Scientist,Summer,DE,"Machine Learning, Python, SQL, Statistics",2.0,2020-06-04
1,2020.0,260000,FT,Technology,Machine Learning Scientist,Autumn,JP,"Deep Learning, Python, Scikit-learn, TensorFlow",3.0,2020-09-26
2,2020.0,85000,FT,Technology,Big Data Engineer,Spring,GB,"ETL, Python, SQL, Spark",2.0,2020-04-06
3,2020.0,20000,FT,Finance,Product Data Analyst,Spring,HN,"Excel, Power BI, SQL, Tableau",2.0,2020-04-10
4,2020.0,150000,FT,Technology,Machine Learning Engineer,Summer,US,"Deep Learning, Python, Scikit-learn, TensorFlow",3.0,2020-08-24


#  Now we had a   pretty good data  to  get the  market  demad for jobs 